In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchaudio
import torch
import numpy as np
import os
import pickle
import re
import torchaudio.transforms as T
import math



torch.manual_seed(1)

In [2]:
torch.__version__

'1.10.0'

In [3]:
file = open('/project/graziul/data/Zone1/2018_08_04/2018_08_04vad_dict.pkl','rb')
vad_dict = pickle.load(file)

In [ ]:
test_key = list(vad_dict.keys())[0]
vad_dict[test_key]

In [ ]:
t = a.get_mfcc()
t.size()

In [4]:
def divide_audio(datafile, div_size = 29): #Divide the audio clip into bits of 1 minute each
#resizes input arrays from (1,feature_length, time) to (div_size,feature_length,time/div_length)
    return np.reshape(datafile,[div_size,datafile.shape[1],datafile.shape[2]//div_size])

class audio_file():
    def __init__(self, name):
        self.name = name
        self.vad_slices = None
        self.frames = None
        self.frames_labels = None
        self.mfcc = None
        self.n_clips = 30
    
    def get_slices(self, vad_dict):
        self.vad_slices = vad_dict[self.name]['pydub'][-24]['nonsilent_slices']
        return self.vad_slices
    
    def get_frames(self):
        ms_2_sample = self.sample_rate/1000
        frames_array = np.zeros(self.mfcc.shape[2])
        #frames_array = np.zeros(180409)

        for v in self.vad_slices:
            start = math.floor(v[0]*ms_2_sample)
            end = math.ceil(v[1]*ms_2_sample)

            for i in range(start,end):
                n = math.floor(i/220)
                j = i%220
                if j <= 110:
                    frames_array[n-2] += 1
                    frames_array[n-1] += 1
                    frames_array[n] += 1
                elif j>=111 and j<=220:
                    frames_array[n-1] += 1
                    frames_array[n] += 1
                elif j>=221 and j<=330:
                    frames_array[n-1] += 1
                    frames_array[n] += 1
                    frames_array[n+1] += 1
                elif j>=331 and j<=440:
                    frames_array[n+1] += 1
                    frames_array[n] += 1
                elif j>=441:
                    frames_array[n+2] += 1
                    frames_array[n+1] += 1
                    frames_array[n] += 1
            
            self.frames = frames_array
            print(self.frame.shape)
            return self.frames
        
    def get_split_frames(self):
        ms_2_sample = self.sample_rate/1000
        frame_arr_list = []
        for j in range(self.n_clips):
            frames_array = np.zeros(self.mfcc.shape[2])
            #frames_array = np.zeros(180409)
            start_idx = j*self.n_clips*self.sample_rate
            end_idx = j*self.n_clips*self.sample_rate
            for v in self.vad_slices:
                start = math.floor(v[0]*ms_2_sample)
                end = math.ceil(v[1]*ms_2_sample)
                if(start > start_idx and end < end_idx):
                    for i in range(start,end):
                        n = math.floor(i/220)
                        j = i%220
                        if j <= 110:
                            frames_array[n-2] += 1
                            frames_array[n-1] += 1
                            frames_array[n] += 1
                        elif j>=111 and j<=220:
                            frames_array[n-1] += 1
                            frames_array[n] += 1
                        elif j>=221 and j<=330:
                            frames_array[n-1] += 1
                            frames_array[n] += 1
                            frames_array[n+1] += 1
                        elif j>=331 and j<=440:
                            frames_array[n+1] += 1
                            frames_array[n] += 1
                        elif j>=441:
                            frames_array[n+2] += 1
                            frames_array[n+1] += 1
                            frames_array[n] += 1
            frame_arr_list.append(np.expand_dims(frames_array,axis = 0))        
        self.frames = np.concatenate(frame_arr_list,axis = 0)
        return self.frames
    
        
    def get_labels(self): 
        self.frames_labels = np.zeros(len(self.frames))
        self.frames_labels[np.where(self.frames>0)] = 1
        return self.frames_labels
    
    def get_split_labels(self):
        self.frames_labels = np.zeros_like(self.frames)
        self.frames_labels[np.where(self.frames>0)] = 1
        return self.frames_labels
        
    def get_mfcc(self): 
        file_name = '/project/graziul/data/Zone1/2018_08_04/' + self.name
        self.waveform, self.sample_rate = torchaudio.load(file_name)
        print(self.waveform.shape, self.sample_rate)
        self.waveform = self.waveform[:,:1800*self.sample_rate] #Clip the file at 1800s
        n_fft = 2048
        win_length = 551
        hop_length = 220
        n_mels = 40
        n_mfcc = 40

        mfcc_transform = T.MFCC(
            sample_rate=self.sample_rate,
            n_mfcc=n_mfcc,
            melkwargs={
              'n_fft': n_fft,
              'n_mels': n_mels,
              'hop_length': hop_length,
              'mel_scale': 'htk',
            }
        )

        self.mfcc = mfcc_transform(self.waveform)
        return self.mfcc
    
    def get_split_mfcc(self):
        file_name = '/project/graziul/data/Zone1/2018_08_04/' + self.name
        self.waveform, self.sample_rate = torchaudio.load(file_name)
        print(self.waveform.shape, self.sample_rate)
        self.waveform = self.waveform[:,:1800*self.sample_rate] #Clip the file at 1800s
        n_clips = self.n_clips
        mfcc_list = []
        n_fft = 2048
        win_length = 551
        hop_length = 220
        n_mels = 40
        n_mfcc = 40
        mfcc_transform = T.MFCC(
                sample_rate=self.sample_rate,
                n_mfcc=n_mfcc,
                melkwargs={
                  'n_fft': n_fft,
                  'n_mels': n_mels,
                  'hop_length': hop_length,
                  'mel_scale': 'htk',
                }
            )
        for i in range(n_clips):
            mfcc_list.append(mfcc_transform(self.waveform[:,i*n_clips*self.sample_rate:(i+1)*n_clips*self.sample_rate]))
        self.mfcc = torch.cat(mfcc_list)
        return self.mfcc

In [ ]:
a = audio_file(test_key)
a.get_slices(vad_dict)
test_mfcc = (a.get_split_mfcc())
print(test_mfcc.shape)
a.get_split_frames()
test_labels= (a.get_split_labels())
print(test_labels.shape)

In [115]:
input_list = []
labels_list = []

for idx,key in enumerate(vad_dict):
    print(idx)
    a = audio_file(key)
    a.get_slices(vad_dict)
    input_list.append(a.get_split_mfcc()) #180409
    a.get_split_frames()
    labels_list.append(a.get_split_labels()) #180409
input_list = torch.cat(input_list)
input_list = torch.transpose(input_list,1,2)
labels_list = torch.from_numpy(np.concatenate(labels_list,axis = 0))
print(input_list.size())
print(labels_list.size())

0
torch.Size([1, 40416944]) 22050
1
torch.Size([1, 40387392]) 22050
2
torch.Size([1, 40374144]) 22050
3
torch.Size([1, 40387392]) 22050
4
torch.Size([1, 40400640]) 22050
5
torch.Size([1, 40400640]) 22050
6
torch.Size([1, 40374144]) 22050
7
torch.Size([1, 40400640]) 22050
8
torch.Size([1, 40374144]) 22050
9
torch.Size([1, 40413888]) 22050
10
torch.Size([1, 40413888]) 22050
11
torch.Size([1, 40400640]) 22050
12
torch.Size([1, 40400640]) 22050
13
torch.Size([1, 40387392]) 22050
14
torch.Size([1, 40413888]) 22050
15
torch.Size([1, 40373568]) 22050
16
torch.Size([1, 40400640]) 22050
17
torch.Size([1, 40413888]) 22050
18
torch.Size([1, 40387392]) 22050
19
torch.Size([1, 40377276]) 22050
20
torch.Size([1, 40387392]) 22050
21
torch.Size([1, 40400640]) 22050
22
torch.Size([1, 40387968]) 22050
23
torch.Size([1, 40400640]) 22050
24
torch.Size([1, 40400640]) 22050
25
torch.Size([1, 40401216]) 22050
26
torch.Size([1, 40400640]) 22050
27
torch.Size([1, 40387392]) 22050
28
torch.Size([1, 40387392]) 2

In [ ]:
f = '/project/graziul/ra/anishk/VAD/Source/Data/data1.pt'
data = torch.load(f)
data = np.transpose(data, (0, 2, 1))

In [ ]:
data.shape

# Initialize LSTM
Pytorch’s LSTM expects all of its inputs to be 3D tensors. The semantics of the axes of these tensors is important. The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input. 

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
input_dim = 40
hidden_dim = 64 
n_layers = 3 

lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

batch_size = 1

hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)

out, hidden = lstm(data, hidden)

In [ ]:
def temp_attention(hidden_feature_map):
    H_maxtemp = torch.max(hidden_feature_map, 1).values
    H_avgtemp = torch.mean(hidden_feature_map, 1)
    H_stdtemp = torch.std(hidden_feature_map, 1)
    H_concattemp = torch.cat([H_maxtemp[None, :], H_avgtemp[None, :], H_stdtemp[None,:]], dim=0)
    return H_concattemp

def freq_attention(hidden_feature_map):
    H_maxfreq = torch.max(hidden_feature_map, 0).values
    H_avgfreq = torch.mean(hidden_feature_map, 0)
    H_stdfreq = torch.std(hidden_feature_map, 0)
    H_concatfreq = torch.cat([H_maxfreq[None, :], H_avgfreq[None, :], H_stdfreq[None,:]], dim=0)
    return H_concatfreq 

def convolve(input,H):
    # Define normalization and relu functions for use after first 3 convolutions
    norm = nn.BatchNorm1d(64, affine=False, track_running_stats=False)
    ReLU = nn.ReLU()

    # 1D Convolution; padding of 5 on both sides to account for ndims change
    conv1 = nn.Conv1d(3,3, kernel_size=11, padding=5)
    output = conv1(input)
    output = norm(output)
    output = ReLU(output)
    
    conv2 = nn.Conv1d(3,5, kernel_size=11, padding=5)
    input = output
    output = conv2(input)
    output = norm(output)
    output = ReLU(output)
    
    conv3 = nn.Conv1d(5,5, kernel_size=11, padding=5)
    input = output
    output = conv3(input)
    output = norm(output)
    output = ReLU(output)
    
    conv4 = nn.Conv1d(5,1, kernel_size=11, padding=5)
    input = output
    H_temp = conv4(input)
    # "Expand/copy" output of last layer (H_temp) to same dims as H
    H_temp = H_temp.expand(-1,64,-1)
    # Sigmoid activation     
    sigmoid = nn.Sigmoid()
    input = H_temp
    H_temp = sigmoid(input)
    H_temp = torch.transpose(H_temp, 1, 2)[0]
    # Merge H_temp and H by element wise summation
    H_prime = torch.stack((H,H_temp))
    H_prime = torch.sum(H_prime,0)
    return H_prime

#H = out[0] ##H is the "Hidden feature map"
#input = temp_attention(H)[None,:] ## batch_size, channels, features
#output = convolve(input,H)

In [ ]:
def display(tensor):
    input = tensor.detach().numpy()
    a = plt.hist(input[0][0], bins = 50)
    b = plt.hist(input[0][1], bins = 50)
    c = plt.hist(input[0][2], bins = 50)
    plt.show()
    d = print('max:',np.max(input))
    e = print('min:',np.min(input))
    range = np.max(input)-np.min(input)
    f = print('range:',range)
    return 

In [ ]:
H = out[0] ##H is the "Hidden feature map"
input = temp_attention(H)[None,:] ## batch_size, channels, features

# Define normalization and relu functions for use after first 3 convolutions
norm = nn.BatchNorm1d(64, affine=False, track_running_stats=False)
ReLU = nn.ReLU()

# 1D Convolution; padding of 5 on both sides to account for ndims change
conv1 = nn.Conv1d(3,3, kernel_size=11, padding=5)
output = conv1(input)
output = norm(output)
output = ReLU(output)

conv2 = nn.Conv1d(3,5, kernel_size=11, padding=5)
input = output
output = conv2(input)
output = norm(output)
output = ReLU(output)


conv3 = nn.Conv1d(5,5, kernel_size=11, padding=5)
input = output
output = conv3(input)
output = norm(output)
output = ReLU(output)

conv4 = nn.Conv1d(5,1, kernel_size=11, padding=5)
input = output
H_temp = conv4(input)
# "Expand/copy" output of last layer (H_temp) to same dims as H
H_temp = H_temp.expand(-1,64,-1)
# Sigmoid activation     
sigmoid = nn.Sigmoid()
input = H_temp
H_temp = sigmoid(input)
H_temp = torch.transpose(H_temp, 1, 2)[0]
# Merge H_temp and H by element wise summation
H_prime = torch.stack((H,H_temp))
H_prime = torch.sum(H_prime,0)
display(H_temp)

In [ ]:
# Same parameters as LSTM1 except for input dim

output = output[0,:,:,:]
output = torch.transpose(output,1,2)
input_dim = 5
hidden_dim = 64 
n_layers = 3 

lstm2 = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

batch_size = 1

hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)

out, hidden = lstm2(output, hidden)

In [ ]:
print(out.size())

In [ ]:
linear1 = nn.Linear(64, 64)
linear1_output = linear1(out)

linear2 = nn.Linear(64, 1)
linear2_output = linear2(linear1_output)

sigmoid = nn.Sigmoid()
input = linear2_output
output_binary = sigmoid(input)

In [ ]:
input = 

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer.zero_grad()
loss_fn(model(input), target).backward()
optimizer.step()

In [103]:
class StackedLSTM(nn.Module):
    def __init__(self):
        super(StackedLSTM, self).__init__()
        self.input_dim1 = 40
        self.input_dim2 = 64 
        self.hidden_dim = 64
        self.n_layers = 3
        self.batch_size = 30
        #(input is of format batch_size, sequence_length, num_features)
        #hidden states should be (num_layers, batch_size, hidden_length)
        self.hidden_state1 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
        self.cell_state1 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
        self.hidden_state2 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
        self.cell_state2 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
        self.lstm1 = nn.LSTM(input_size = self.input_dim1, hidden_size = self.hidden_dim, num_layers = self.n_layers, batch_first=True) #should be True
        self.lstm2 = nn.LSTM(input_size = self.input_dim2, hidden_size = self.hidden_dim, num_layers = self.n_layers, batch_first=True) #should be True
        self.lstm2_out = None 
        self.hidden = None
        #self.flatten = nn.Flatten()
        self.convolve1d = nn.Sequential(
            nn.Conv1d(3,3, kernel_size=11, padding=5),
            nn.BatchNorm1d(64, affine=False, track_running_stats=False),
            nn.ReLU(),
            nn.Conv1d(3,5, kernel_size=11, padding=5),
            nn.BatchNorm1d(64, affine=False, track_running_stats=False),
            nn.ReLU(),
            nn.Conv1d(5,5, kernel_size=11, padding=5),
            nn.BatchNorm1d(64, affine=False, track_running_stats=False),
            nn.ReLU(),
            nn.Conv1d(5,1, kernel_size=11, padding=5)
        )
        self.output_stack = nn.Sequential(
            nn.Linear(64, 64),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

#     def create_rand_hidden1(self):
#         self.hidden_state1 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
#         self.cell_state1 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
#         return (self.hidden_state1, self.cell_state1)

    def temp_attention(self, data):
        H, hidden = self.lstm1(data, (self.hidden_state1, self.cell_state1))
        H_maxtemp = torch.unsqueeze(torch.max(H, -1).values,2)
        H_avgtemp = torch.unsqueeze(torch.mean(H, -1),2)
        H_stdtemp = torch.unsqueeze(torch.std(H, -1),2)
        H_concattemp = torch.cat([H_maxtemp, H_avgtemp,H_stdtemp], dim=2)
        H_concattemp = torch.transpose(H_concattemp, 1,2)
        return H_concattemp,H 
    
    def convolve1(self, data):
        H_concattemp,H = self.temp_attention(data)
        H_temp = self.convolve1d(H_concattemp)
        # "Expand/copy" output of last layer (H_temp) to same dims as H
        H_temp = H_temp.expand(-1,64,-1)
        # Sigmoid activation     
        sigmoid = nn.Sigmoid()
        my_input = H_temp
        H_temp = sigmoid(my_input)
        H_temp = torch.transpose(H_temp, 1, 2)
        # Merge H_temp and H by element wise summation
        H_prime = torch.stack((H,H_temp))
        H_prime = torch.sum(H_prime,0)
        return H_prime
        
#     def create_rand_hidden2(self):
#         self.hidden_state2 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
#         self.cell_state2 = torch.randn(self.n_layers, self.batch_size, self.hidden_dim)
#         return (self.hidden_state2, self.cell_state2)  
    
#     def freq_attention(hidden_feature_map):
#         H_maxfreq = torch.max(hidden_feature_map, 0).values
#         H_avgfreq = torch.mean(hidden_feature_map, 0)
#         H_stdfreq = torch.std(hidden_feature_map, 0)
#         H_concatfreq = torch.cat([H_maxfreq[None, :], H_avgfreq[None, :], H_stdfreq[None,:]], dim=0)
#         return H_concatfreq 

    def forward(self, data):
        input1 = self.convolve1(data)
        lstm2_out, hidden = self.lstm2(input1, (self.hidden_state2, self.cell_state2))
        self.output = self.output_stack(lstm2_out)
        return torch.squeeze(self.output)
        

In [41]:
class WeightedFocalLoss(nn.Module):
    "Non weighted version of Focal Loss"
    def __init__(self, alpha=.25, gamma=1):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1-alpha]).cuda()
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        targets = targets.type(torch.long)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return F_loss.mean()


In [110]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=1,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, inputs, targets):
        #BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        #return torch.mean(BCE_loss)
        ce_loss = F.binary_cross_entropy_with_logits(inputs, targets,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [ ]:
labels_list = torch.tensor(labels_list[:31])
labels_list.size()

In [116]:
model = StackedLSTM().to(device)
loss_fn = FocalLoss()

In [11]:
device = torch.device("cpu")

In [20]:
input_list = input_list[:30]
labels_list = labels_list[:30]

In [19]:
input_list.size()

torch.Size([1050, 3007, 40])

In [117]:
training_steps = 50
num_samples = input_list.size()[0]//30
idx = 0
for step in range(training_steps):
    input_batch = input_list[idx*30:(idx+1)*30]
    labels_batch = labels_list[idx*30:(idx+1)*30]
    idx = (idx+1)%num_samples
    print(step)
    optimizer = torch.optim.SGD(model.parameters(), lr=1, momentum=0.9)
    optimizer.zero_grad()
    output_hat = model(input_batch)
    loss = loss_fn(output_hat, labels_batch)
    loss.backward()
    print(loss)
    optimizer.step()

0
tensor(0.6065, dtype=torch.float64, grad_fn=<MeanBackward0>)
1
tensor(0.5656, dtype=torch.float64, grad_fn=<MeanBackward0>)
2
tensor(0.5277, dtype=torch.float64, grad_fn=<MeanBackward0>)
3
tensor(0.4938, dtype=torch.float64, grad_fn=<MeanBackward0>)
4
tensor(0.4643, dtype=torch.float64, grad_fn=<MeanBackward0>)
5
tensor(0.4400, dtype=torch.float64, grad_fn=<MeanBackward0>)
6
tensor(0.4206, dtype=torch.float64, grad_fn=<MeanBackward0>)
7
tensor(0.4056, dtype=torch.float64, grad_fn=<MeanBackward0>)
8
tensor(0.3941, dtype=torch.float64, grad_fn=<MeanBackward0>)
9
tensor(0.3853, dtype=torch.float64, grad_fn=<MeanBackward0>)
10
tensor(0.3786, dtype=torch.float64, grad_fn=<MeanBackward0>)
11
tensor(0.3735, dtype=torch.float64, grad_fn=<MeanBackward0>)
12
tensor(0.3694, dtype=torch.float64, grad_fn=<MeanBackward0>)
13
tensor(0.3662, dtype=torch.float64, grad_fn=<MeanBackward0>)
14
tensor(0.3637, dtype=torch.float64, grad_fn=<MeanBackward0>)
15
tensor(0.3616, dtype=torch.float64, grad_fn=<Me

In [118]:
outputs = (np.round(output_hat.detach().numpy()))
labels = (labels_list.detach().numpy())
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, classification_report
print(accuracy_score(labels,outputs))

ValueError: Found input variables with inconsistent numbers of samples: [1050, 30]

In [ ]:
filepath = '/project/graziul/ra/ajays/trimmed_threshold_plots/frame_error_rate_52.png'
import matplotlib.image as img
my_img = img.imread(filepath)
plt.imshow(my_img)